In [23]:
import sys
import gc

In [19]:
x = [1, 2, 3]
print sys.getrefcount(x)

2


In [20]:
y = x
print sys.getrefcount(x)

3


In [4]:
del(y)
print sys.getrefcount(x)

2


In [6]:
import gc
print gc.garbage

[]


In [15]:
#cyclic references
import gc
import pprint

class Graph(object):
    def __init__(self, name):
        self.name = name
        self.next = None
    def set_next(self, next):
        print 'Linking nodes %s.next = %s' % (self, next)
        self.next = next
    def __repr__(self):
        return '%s(%s)' % (self.__class__.__name__, self.name)

# Construct a graph cycle
one = Graph('one')
two = Graph('two')
three = Graph('three')
one.set_next(two)
two.set_next(three)
three.set_next(one)

print

# Remove references to the graph nodes in this module's namespace
one = two = three = None

# Show the effect of garbage collection
for i in range(2):
    print 'Collecting %d ...' % i
    n = gc.collect()
    print 'Unreachable objects:', n
    print 'Remaining Garbage:', 
    pprint.pprint(gc.garbage)
    print

Linking nodes Graph(one).next = Graph(two)
Linking nodes Graph(two).next = Graph(three)
Linking nodes Graph(three).next = Graph(one)

Unreachable objects: 38
Remaining Garbage:[]

Unreachable objects: 0
Remaining Garbage:[]



### If Graph has a __del__() method, however, the garbage collector cannot break the cycle.

###Because...

* more than one object in the cycle has a finalizer method
* the order in which the objects need to be finalized and then garbage collected cannot be determined
* so the garbage collector plays it safe and keeps the objects.

In [25]:
import gc
import pprint

class Graph(object):
    def __init__(self, name):
        self.name = name
        self.next = None
    def set_next(self, next):
        print '%s.next = %s' % (self, next)
        self.next = next
    def __repr__(self):
        return '%s(%s)' % (self.__class__.__name__, self.name)
    def __del__(self):
        print '%s.__del__()' % self

# Construct a graph cycle
one = Graph('one')
two = Graph('two')
three = Graph('three')
one.set_next(two)
two.set_next(three)
three.set_next(one)

# Remove references to the graph nodes in this module's namespace
one = two = three = None

# Show the effect of garbage collection
print 'Collecting...'
n = gc.collect()
print 'Unreachable objects:', n
print 'Remaining Garbage:', 
pprint.pprint(gc.garbage)

Graph(one).next = Graph(two)
Graph(two).next = Graph(three)
Graph(three).next = Graph(one)
Collecting...
Unreachable objects: 23
Remaining Garbage:[Graph(one), Graph(two), Graph(three)]



### the Graph instances can be collected only when the cycle is broken

In [26]:
# Break the cycle
print
print 'Breaking the cycle'
gc.garbage[0].set_next(None)
print 'Removing references in gc.garbage'
del gc.garbage[:]

# Now the objects are removed
print
print 'Collecting...'
n = gc.collect()
print 'Unreachable objects:', n
print 'Remaining Garbage:', 
pprint.pprint(gc.garbage)


Breaking the cycle
Graph(one).next = None
Removing references in gc.garbage
Graph(two).__del__()
Graph(three).__del__()
Graph(one).__del__()

Collecting...
Unreachable objects: 9
Remaining Garbage:[]



#### Note: gc.garbage holds a reference to the objects from the previous garbage collection run. It needs to be cleared out after the cycle is broken